In [55]:
# Import of the training data

from google.colab import files
uploaded = files.upload()

# Import of moduls

!pip install wikidata --quiet

import matplotlib.pyplot as plt
import pandas as pd
from wikidata.client import Client

from tqdm.auto import tqdm
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Loading the data
train_df = pd.read_csv('[MNLP 2025 HW1] train set [PUBLIC] - train_cleaned.tsv', sep='\t')

Saving [MNLP 2025 HW1] train set [PUBLIC] - train_cleaned.tsv to [MNLP 2025 HW1] train set [PUBLIC] - train_cleaned (2).tsv


In [42]:
def extract_entity_id(url):
    return url.strip().split("/")[-1]

In [43]:
def get_claims(wikidata_url):
  # Extract the id of the item
    entity_id = extract_entity_id(wikidata_url)

    # Extract information of the item
    client = Client()
    item = client.get(entity_id, load=True)

    # Extract the claims of the item
    claims = item.data.get("claims", {})

    return claims

In [52]:
def get_first_line_with_statement(df, statement):

  founded = False

  for row in range(0, len(df)+1):

    if founded == False :

      wikidata_url = df.iloc[row,0]
      claims = get_claims(wikidata_url)

      # Extract the statement :
      unicode_statements = claims.get(statement, {})
      if unicode_statements:
        print(f"The first line with the statement {statement} is : ", row)
        return row

  if founded == False:
    print(f"No row in the dataset, has the statement {statement}")


get_first_line_with_statement(df=train_df, statement="P571")

The first line with the statement P571 is :  58


58

In [46]:
wikidata_url = train_df.iloc[58,0]
claims = get_claims(wikidata_url)

# Extract the statement "P495" : Country of origin
unicode_statements = claims.get("P571", [])
if unicode_statements:
  for statement in unicode_statements:
     mainsnak = statement.get("mainsnak", {})
     datavalue = mainsnak.get("datavalue", {})
     value = datavalue.get("value", {})
     time = value.get('time', {})
     print(time)

+1909-00-00T00:00:00Z


In [96]:
import numpy as np

def extract_year_from_date(date):
    return date[1:5]


def get_creation_date(wikipedia_url):
    """
    Function that extract the creation date of an item.

    Arguments:
      wikipedia_url : str

    Returns:
      creation_date : str
    """

    creation_date = np.nan
    date_of_birth = np.nan
    date_of_publication = np.nan
    claims = get_claims(wikipedia_url)

    # Extract the statement "P571" : Creation Date
    unicode_statements = claims.get("P571", [])
    if unicode_statements:
        for statement in unicode_statements:
            mainsnak = statement.get("mainsnak", {})
            datavalue = mainsnak.get("datavalue", {})
            value = datavalue.get("value", {})
            time = value.get('time')
            if time:
                creation_date = extract_year_from_date(time)


    # Extract the statement "P569" : Date of Birth
    unicode_statements = claims.get("P569", [])
    if unicode_statements:
        for statement in unicode_statements:
            mainsnak = statement.get("mainsnak", {})
            datavalue = mainsnak.get("datavalue", {})
            value = datavalue.get("value", {})
            time = value.get('time')
            if time:
                date_of_birth = extract_year_from_date(time)


    # Extract the statement "P577" : Date of Publication
    unicode_statements = claims.get("P577", [])
    if unicode_statements:
        for statement in unicode_statements:
            mainsnak = statement.get("mainsnak", {})
            datavalue = mainsnak.get("datavalue", {})
            value = datavalue.get("value", {})
            time = value.get('time')
            if time:
                date_of_publication = extract_year_from_date(time)

    return creation_date, date_of_birth, date_of_publication

In [108]:
train_df['creation_date'] = None
train_df['date_of_birth'] = None
train_df['date_of_publication'] = None

subset = range(0, 3)
date_informations = train_df.loc[subset, "item"].apply(lambda url: get_creation_date(url))

creation_dates_df = pd.DataFrame(date_informations.tolist(), columns=['creation_date', 'date_of_birth', 'date_of_publication'])

train_df.loc[subset, 'creation_date'] = creation_dates_df['creation_date']
train_df.loc[subset, 'date_of_birth'] = creation_dates_df['date_of_birth']
train_df.loc[subset, 'date_of_publication'] = creation_dates_df['date_of_publication']

train_df

,item,name,description,type,category,subcategory,label,creation_date,date_of_birth,date_of_publication
0,http://www.wikidata.org/entity/Q306,Sebastián Piñera,Chilean entrepreneur and politician (1949–2024),entity,politics,politician,cultural exclusive,NaN,1949,NaN
1,http://www.wikidata.org/entity/Q12735,John Amos Comenius,"Czech teacher, educator, philosopher and write...",entity,politics,politician,cultural representative,NaN,1592,NaN
2,http://www.wikidata.org/entity/Q1752,Macrinus,Roman emperor from 217 to 218,entity,politics,politician,cultural representative,NaN,0165,NaN
3,http://www.wikidata.org/entity/Q1639,Lamine Diack,Senegalese sports manager (1933–2021),entity,politics,politician,cultural representative,None,None,None
4,http://www.wikidata.org/entity/Q9588,Richard Nixon,President of the United States from 1969 to 1974,entity,politics,politician,cultural representative,None,None,None
...,...,...,...,...,...,...,...,...,...,...
6246,http://www.wikidata.org/entity/Q321103,Bühl,"quarter of Tübingen, Baden-Württemberg, Germany",entity,geography,neighborhood,cultural exclusive,None,None,None
6247,http://www.wikidata.org/entity/Q338167,Tenderloin,area of New York City during the late 19th and...,entity,geography,neighborhood,cultural exclusive,None,None,None
6248,http://www.wikidata.org/entity/Q66991,Schinznach-Dorf,former municipality and current district of Sc...,entity,geography,neighborhood,cultural exclusive,None,None,None
6249,http://www.wikidata.org/entity/Q66922,Ependes,village and former municipality in Bois-d'Amon...,entity,geography,neighborhood,cultural exclusive,None,None,None


In [114]:
train_df['creation_date'] = None
train_df['date_of_birth'] = None
train_df['date_of_publication'] = None

subset = range(4799, 5400)
date_informations = train_df.loc[subset, "item"].apply(lambda url: get_creation_date(url))

creation_dates_df = pd.DataFrame(date_informations.tolist(), columns=['creation_date', 'date_of_birth', 'date_of_publication'])

train_df.loc[subset, 'creation_date'] = creation_dates_df['creation_date']
train_df.loc[subset, 'date_of_birth'] = creation_dates_df['date_of_birth']
train_df.loc[subset, 'date_of_publication'] = creation_dates_df['date_of_publication']

#print(train_df[train_df['date_of_publication'] is not None])
print(train_df[train_df['date_of_publication'].notna()])

Empty DataFrame
Columns: [item, name, description, type, category, subcategory, label, creation_date, date_of_birth, date_of_publication]
Index: []
